Some functions that are needed in this process are provided. You should run this code before other ones.

In [3]:
import math

def Chi(obs,exp):
    s=[]
    for i in range(len(obs)):
        if exp[i] !=0: #just an idea
            ss = ((obs[i] - exp[i])**2)/exp[i]
            s.append(ss)
    
    return sum(s)

def MSE(obs,exp):
    s=[]
    for i in range(len(obs)):
        ss = (obs[i] - exp[i])**2
        s.append(ss)
        mse = sum(s)/len(obs)
        
    return(mse)

def MAE(obs,exp):
    s=[]
    for i in range(len(obs)):
        ss = abs(obs[i] - exp[i])
        s.append(ss)
        mae = sum(s)/len(obs)
        
    return(mae)

def RMSE(obs,exp):
    s=[]
    for i in range(len(obs)):
        ss = (obs[i] - exp[i])**2
        s.append(ss)
        mse = sum(s)/len(obs)
        rmse = math.sqrt(mse)
        
    return(rmse)

In [21]:

"""
@author: taha

"""
import numpy as np
from okada_wrapper import dc3dwrapper
#import scipy.optimize.nnls as Snnls
from scipy.optimize import nnls as Snnls
import math
#from scipy.sparse import rand
from scipy.optimize import lsq_linear
from scipy.sparse.linalg import lsmr
from numpy.linalg import matrix_rank
from numpy import linalg 
from scipy.linalg import norm
from scipy.stats import chisquare

### explanation for units
## Distance: km

## Geometry of patches


x0 = 404
y0 = 4222
dx = 1.0 # spacing along X/row
dy = 1.0 # spacing along Y/column
nrow = 20 # number of row dip 
ncol = 40 # number of column strike

## patch parameters
alpha = 0.6666666
depth_top = 0 # km  #it will be 1.2 to 20 km for instance
dip = 80
strike = 76 # degree
rake = 0 # degree

a = -1.0*(strike-90.)*np.pi/180. # rotation angle


########## Reading data

### The format that is used in this code is "tab delimited". So, you need to first convert your data to this format.
## or change the code :) 

## read InSAR DESCENDING LOS observation
f_des = open('descending_example_nevada.dat','r')   
line = f_des.readline().strip() # skip the first line
obs_x_utm_des = []
obs_y_utm_des = []
obs_los_utm_des = []

while True:
    line = f_des.readline().strip()
    if line=='':
        #print ('break')
        break
    raw = line.split('	')
    obs_x_utm_des.append(float(raw[0]))	
    obs_y_utm_des.append(float(raw[1]))
    obs_los_utm_des.append(float(raw[2]))
f_des.close()

## read InSAR ASCENDING LOS observation
f_asc = open('ascending_example_nevada.dat','r')   
line = f_asc.readline().strip() # skip the first line (look at the example data)
obs_x_utm_asc = []
obs_y_utm_asc = []
obs_los_utm_asc = []

while True:
    line = f_asc.readline().strip()
    if line=='':
        #print ('break')
        break
    raw = line.split('	')
    obs_x_utm_asc.append(float(raw[0]))	
    obs_y_utm_asc.append(float(raw[1]))
    obs_los_utm_asc.append(float(raw[2]))
f_asc.close()

######################## Concatenating vectors
obs_x_utm = np.concatenate((obs_x_utm_des, obs_x_utm_asc))
obs_y_utm = np.concatenate((obs_y_utm_des, obs_y_utm_asc))
obs_los_utm = np.concatenate((obs_los_utm_des, obs_los_utm_asc))

n_obs_des = len(obs_los_utm_des)
n_obs_asc = len(obs_los_utm_asc)
n_obs = len(obs_los_utm)

#######################
#converted coordinates to okada coordinate: Conversion from utm coordinates to okada coordinates for obs

x_tmp_des=[] 
y_tmp_des=[]

for i in range(n_obs_des):
    xxmpd = (obs_x_utm_des[i] - x0)*np.cos(-a) - (obs_y_utm_des[i] - y0)*np.sin(-a)
    yympd = (obs_x_utm_des[i] - x0)*np.sin(-a) + (obs_y_utm_des[i] - y0)*np.cos(-a)
    x_tmp_des.append(xxmpd)
    y_tmp_des.append(yympd)

#converted coordinates to okada coordinate: Conversion from utm coordinates to okada coordinates for obs
    
x_tmp_asc=[] 
y_tmp_asc=[]

for i in range(n_obs_asc):
    xxmpa = (obs_x_utm_asc[i] - x0)*np.cos(-a) - (obs_y_utm_asc[i] - y0)*np.sin(-a)
    yympa = (obs_x_utm_asc[i] - x0)*np.sin(-a) + (obs_y_utm_asc[i] - y0)*np.cos(-a)
    x_tmp_asc.append(xxmpa)
    y_tmp_asc.append(yympa)
    
#######################

## convert disp to LOS. use the equation in Hu, J., Li, Z., et al., 2012

#it is descending incidence and heading angles

inc_d = 34.0 * np.pi/180. # rad
heading_d = -166.71 * np.pi/180.0

#it is ascending incidence and heading angles

inc_a = 3.397904225855917e+01 * np.pi/180. # rad
heading_a = -1.324459955550515e+01 * np.pi/180.0

#######################

dip_rad = dip * np.pi/180.0 # rad

### calculated Green's function for DESCENDING

G_des = np.zeros((n_obs_des, nrow*ncol)) # for strike-slip motion
GG_des = np.zeros((n_obs_des, nrow*ncol)) # for dip-slip motion


# G matrix for strike-slip parts

for i in range(n_obs_des):
    ob_des = [x_tmp_des[i], y_tmp_des[i], 0] # at surface
    for r in range(nrow): # dip #decrease the number of rows or spacing dy
        for c in range(ncol): #strike
            al1 = c*dx  
            al2 = al1 + dx
            aw1 = -r*dy  
            aw2 = aw1 - dy
            


            
            success,u,grad_u = dc3dwrapper(alpha, ob_des, depth_top, dip, [al1, al2], [aw1, aw2], [1, 0, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 
            
            
            u1 = u[0] * 1.e6 
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation

            los_d_sim = u3 * np.cos(inc_d) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_d) * np.sin(inc_d))
            G_des[i][ncol*r+c] = los_d_sim

# G matrix (here GG) for dip-slip parts.

for i in range(n_obs_des):
    ob_des = [x_tmp_des[i], y_tmp_des[i], 0] # at surface
    for r in range(nrow): 
        for c in range(ncol): #strike
            al1 = c*dx  #x0+...
            al2 = al1 + dx
            aw1 = -r*dy  #y0+...
            aw2 = aw1 - dy
            


            
            success,u,grad_u = dc3dwrapper(alpha, ob_des, depth_top, dip, [al1, al2], [aw1, aw2], [0, 1, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 
            
            
            u1 = u[0] * 1.e6 
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation
 
            los_d_sim = u3 * np.cos(inc_d) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_d) * np.sin(inc_d))
            GG_des[i][ncol*r+c] = los_d_sim

#concatenate those G into one G           
G_new_des = np.hstack((G_des,GG_des))   # its size should be (n_obs * 2 nrow ncol)

############## END of G for DESCENDING

## G for Ascending

G_asc = np.zeros((n_obs_asc, nrow*ncol)) # for strike-slip motion
GG_asc = np.zeros((n_obs_asc, nrow*ncol)) # for dip-slip motion

for i in range(n_obs_asc):
    ob_asc = [x_tmp_asc[i], y_tmp_asc[i], 0] # at surface
    for r in range(nrow):
        for c in range(ncol): #strike
            al1 = c*dx  #x0+...
            al2 = al1 + dx
            aw1 = -r*dy  #y0+...
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob_asc, depth_top, dip, [al1, al2], [aw1, aw2], [1, 0, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 
            #print (u)
            
            u1 = u[0] * 1.e6 
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation

            los_a_sim = u3 * np.cos(inc_a) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_a) * np.sin(inc_a))
            G_asc[i][ncol*r+c] = los_a_sim

# G matrix (here GG) for dip-slip parts.

for i in range(n_obs_asc):
    ob_asc = [x_tmp_asc[i], y_tmp_asc[i], 0] # at surface
    for r in range(nrow): 
        for c in range(ncol): #strike
            al1 = c*dx  
            al2 = al1 + dx
            aw1 = -r*dy  
            aw2 = aw1 - dy


            success,u,grad_u = dc3dwrapper(alpha, ob_asc, depth_top, dip, [al1, al2], [aw1, aw2], [0, 1, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 
            
            u1 = u[0] * 1.e6
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation
 
            los_a_sim = u3 * np.cos(inc_a) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_a) * np.sin(inc_a))
            GG_asc[i][ncol*r+c] = los_a_sim

#concatenate those G into one G           
G_new_acs = np.hstack((G_asc,GG_asc))   # its size should be (n_obs * 2 nrow ncol)

#### Merging Green functions from different des and asc

G_new = np.vstack((G_new_des,G_new_acs))  # should be (n_obs_des + n_obs_asc)*(2*nrow*ncol)

                       
### Create roughness/smoothness matrix I
### our constrain: e.g., v1-2*v2+v3=0

I = np.zeros((nrow*ncol, nrow*ncol)) # for strike-slip motion
I1 = np.zeros((nrow*ncol, nrow*ncol)) # for dip-slip motion


for r in range(nrow*ncol):
    for c in range(nrow*ncol):
        if (r==c):
            I[r][c] = -2
        elif (c>r):
            I[r][r+1] = 1
        elif (c<r):
            I[r][r-1] = 1



for r in range(nrow*ncol):
    for c in range(nrow*ncol):
        if (c==r*ncol):
            I[c][c-1] = 0
            I[c-1][c] = 0
            
        
###### finished            

####the other section (vertical) I1


for r in range(nrow*ncol):
    for c in range(nrow*ncol):
        if (r==c):
            I1[r][c] = -2
        elif (c<r) and (r>=ncol):
            I1[r][r-ncol] = 1
        elif (c>r) and (c+ncol<=nrow*ncol):
            I1[r][r+ncol] = 1

####finished vertical part

I_ss = np.vstack((I,I1))
I_ds = np.vstack((I,I1))

zero1 = np.zeros((2*nrow*ncol, nrow*ncol))     

I_new_ss = np.hstack((I_ss, zero1)) #smoothness for strike-slip

I_new_ds = np.hstack((zero1, I_ds)) #smoothness for dip-slip
I_new = np.vstack((I_new_ss, I_new_ds)) #smoothness matrix 
################## END of Smoothining

#initializing the bounding value for LS. This is based on nature of problem. 
#You can apply simple least square instead of bounding LS. 

lb = np.zeros(2*nrow*ncol)
ub = np.zeros(2*nrow*ncol)


for p in range(nrow*ncol): # ub = [0.002,.....0.002,0,0,0...0] #lb = [0,0,0....0, -0.0005,....-0.0005]
    ub[p] = 0.002
    
for k in range(nrow*ncol,2*nrow*ncol):
    lb[k] = -0.0002

           
### inversion using different e (somepalce it's called lambda).
### it's kind of the weight for the smoothness/roughness 

e_min = 1.0
e_max = 1.0e11
N = 20 ## You divide the e's range and apply each of those steps in each LS.
power = np.linspace(math.log10(e_min), math.log10(e_max), N)
e = [0.0] * N
for i in range(N):
    e[i] = math.pow(10.0, power[i])

misfit = [0.0] * N
chi = [0.0] * N
residual_obs = [0.0] * N
mean_abs_err = [0.0] * N
mean_sqr_err = [0.0] * N
root_mean_sqr_err = [0.0] * N
#p_values = [0.0] * N
### save result to file
f_out = open('eq_Patches_smoothness_misfit_nevada_N%d_strike%d_dip%d.dat' %(N,strike,dip), 'w')
f_out.write("smoothness\tMAE\tMSE\tmisfit\tRMSE\n")  
#CX = D new big green function (C) and observation Vector (D)
for l in range(N):
    ### construct new matrix C
    C = np.zeros((n_obs + 4*nrow*ncol, 2*nrow*ncol))
    for i in range(n_obs):
        for j in range(2*nrow*ncol):
            C[i][j] = G_new[i][j]

    for i in range(4*nrow*ncol):
        for j in range(2*nrow*ncol):
            C[i+n_obs][j] = e[l] * I_new[i][j]


        

    D =[0.0]*(n_obs + 4*nrow*ncol)
    for i in range(n_obs):
        D[i] = obs_los_utm[i] 


    tmp = lsq_linear(C, D, bounds=(lb, ub), lsmr_tol='auto', max_iter=200, verbose=1)
    
    #print ('smoothness: ', e[l], ' misfit: ', norm(tmp.fun))    

    misfit[l] = norm(tmp.fun) # normr : norm(D-Cx) tmp[1] norm(tmp.fun)
    residual_obs[l] = tmp.fun + D

    mean_abs_err[l] = MAE(residual_obs[l],D)
    mean_sqr_err[l] = MSE(residual_obs[l],D)
    root_mean_sqr_err[l] = RMSE(residual_obs[l], D)

    f_out.write("%.2e\t%f\t%f\t%f\t%f\n" % (e[l], mean_abs_err[l], mean_sqr_err[l], misfit[l], root_mean_sqr_err[l])) 

f_out.close()



In the next code, you plot "L-curve". In most of the cases, you need to modify the file that is created in previous step and then plot. However, you read the previous made file, plot "smoothness-RMS error" and choose the best smoothness parameter.

In [22]:
#https://stackoverflow.com/questions/2018178/finding-the-best-trade-off-point-on-a-curve
#https://stackoverflow.com/questions/39840030/distance-between-point-and-a-line-from-two-points

N = 20

dip = 80
strike = 76 # degree

#read file
f = open('eq_Patches_smoothness_misfit_nevada_N%d_strike%d_dip%d.dat' %(N,strike,dip),'r')
raw = f.readline().strip().split() # skip the first line

rough = []
misfit = []

while True:
    line = f.readline().strip()
    if line=='':
        print ('break')
        break
    raw = line.split('	')

    rough.append(math.log10(float(raw[0])))	
    misfit.append(float(raw[4]))


## calculate the distance
## Points L1 and L2 define the line L, calculate the distance from point p to line L

L1 = np.asarray([misfit[0], rough[0]])
L2 = np.asarray([misfit[-1], rough[-1]]) 

d = [0] * len(rough)
dmax = 0.0
dmax_x = 0.0
dmax_y = 0.0
for i in range(len(rough)):
    p = np.asarray([misfit[i], rough[i]]) 
    d=abs(np.cross(L2-L1,p-L1)/np.linalg.norm(L2-L1))
    if d > dmax:
        dmax = d
        dmax_x = misfit[i]
        dmax_y = rough[i]

print (dmax_x, dmax_y)

## plot
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(1,1)
gs.update(wspace=0.0, hspace=0.0)

ax = fig.add_subplot(gs[0])

plt.plot(misfit, rough, '.-') 
ax.scatter(dmax_x, dmax_y, marker='o', c='red', s=40) 

#ax.set_yscale('log')

plt.xlabel('RMS (mm)')
plt.ylabel('log(smoothness)')

fig.savefig('eq_L_curve_corner_nevada_Desc+Asc_N%d.png' %(N), dpi=300, bbox_inches='tight')



You now, know the best smoothness parameter for your model. Thus, you choose that one and invert your data using that smoothness (e).

In [23]:
### 2 constrains: smoothness, all unknown parameters have the same sign (positive or negative)
## Below links are helpful infomation for the understanding of the inversion. 

#https://en.wikipedia.org/wiki/Tikhonov_regularization
#https://scicomp.stackexchange.com/questions/10671/tikhonov-regularization-in-the-non-negative-least-square-nnls-pythonscipy/10673
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.nnls.html
#https://en.wikipedia.org/wiki/Non-negative_least_squares

import numpy as np
from okada_wrapper import dc3dwrapper
#import scipy.optimize.nnls as Snnls
from scipy.optimize import nnls as Snnls
import math
#from scipy.sparse import rand
from scipy.optimize import lsq_linear
from scipy.sparse.linalg import lsmr
from numpy.linalg import matrix_rank
from numpy import linalg 
from scipy.linalg import norm
from scipy.stats import chisquare

### explanation for units
## Distance: km

## Geometry of patches

x0 = 404
y0 = 4222
dx = 1.0 
dy = 1.0 
nrow = 20  
ncol = 40 

## patch parameters
alpha = 0.6666666
depth_top = 0 # km  
dip = 80
strike = 76 # degree
rake = 0 # degree


e = 6.16e+05  ## this is the example. you need to check yout file.

a = -1.0*(strike-90.)*np.pi/180. # rotation angle


########## Reading data

## read InSAR DESCENDING LOS observation
f_des = open('descending_example_nevada.dat','r')   
line = f_des.readline().strip() # skip the first line
obs_x_utm_des = []
obs_y_utm_des = []
obs_los_utm_des = []

while True:
    line = f_des.readline().strip()
    if line=='':
        #print ('break')
        break
    raw = line.split('	')
    obs_x_utm_des.append(float(raw[0]))	
    obs_y_utm_des.append(float(raw[1]))
    obs_los_utm_des.append(float(raw[2]))
f_des.close()

## read InSAR ASCENDING LOS observation
f_asc = open('ascending_example_nevada.dat','r')   
line = f_asc.readline().strip() # skip the first line
obs_x_utm_asc = []
obs_y_utm_asc = []
obs_los_utm_asc = []

while True:
    line = f_asc.readline().strip()
    if line=='':
        #print ('break')
        break
    raw = line.split('	')
    obs_x_utm_asc.append(float(raw[0]))	
    obs_y_utm_asc.append(float(raw[1]))
    obs_los_utm_asc.append(float(raw[2]))
f_asc.close()

######################## Concatenating vectors
obs_x_utm = np.concatenate((obs_x_utm_des, obs_x_utm_asc))
obs_y_utm = np.concatenate((obs_y_utm_des, obs_y_utm_asc))
obs_los_utm = np.concatenate((obs_los_utm_des, obs_los_utm_asc))

n_obs_des = len(obs_los_utm_des)
n_obs_asc = len(obs_los_utm_asc)
n_obs = len(obs_los_utm)

#######################

x_tmp_des=[] #converted coordinates to okada coordinate: Conversion from utm coordinates to okada coordinates for obs
y_tmp_des=[]

for i in range(n_obs_des):
    xxmpd = (obs_x_utm_des[i] - x0)*np.cos(-a) - (obs_y_utm_des[i] - y0)*np.sin(-a)
    yympd = (obs_x_utm_des[i] - x0)*np.sin(-a) + (obs_y_utm_des[i] - y0)*np.cos(-a)
    x_tmp_des.append(xxmpd)
    y_tmp_des.append(yympd)
    
x_tmp_asc=[] #converted coordinates to okada coordinate: Conversion from utm coordinates to okada coordinates for obs
y_tmp_asc=[]

for i in range(n_obs_asc):
    xxmpa = (obs_x_utm_asc[i] - x0)*np.cos(-a) - (obs_y_utm_asc[i] - y0)*np.sin(-a)
    yympa = (obs_x_utm_asc[i] - x0)*np.sin(-a) + (obs_y_utm_asc[i] - y0)*np.cos(-a)
    x_tmp_asc.append(xxmpa)
    y_tmp_asc.append(yympa)
    
#######################

## convert disp to LOS. use the equation in Hu, J., Li, Z., et al., 2012


#it is descending angles
inc_d = 34.0 * np.pi/180. # rad
heading_d = -166.71 * np.pi/180.0

#it is ascending angles
inc_a = 3.397904225855917e+01 * np.pi/180. # rad
heading_a = -1.324459955550515e+01 * np.pi/180.0

dip_rad = dip * np.pi/180.0 # rad
### calculated Green's function

G = np.zeros((n_obs, nrow*ncol)) # for strike-slip motion
GG = np.zeros((n_obs, nrow*ncol)) # for dip-slip motion

# G matrix for strike-slip parts

### calculated Green's function for DESCENDING

G_des = np.zeros((n_obs_des, nrow*ncol)) # for strike-slip motion
GG_des = np.zeros((n_obs_des, nrow*ncol)) # for dip-slip motion

# G matrix for strike-slip parts

for i in range(n_obs_des):
    ob_des = [x_tmp_des[i], y_tmp_des[i], 0] # at surface
    for r in range(nrow): 
        for c in range(ncol): 
            al1 = c*dx  
            al2 = al1 + dx
            aw1 = -r*dy  
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob_des, depth_top, dip, [al1, al2], [aw1, aw2], [1, 0, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 
            #print (u)
            
            u1 = u[0] * 1.e6 
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
 
            los_d_sim = u3 * np.cos(inc_d) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_d) * np.sin(inc_d))
            G_des[i][ncol*r+c] = los_d_sim

# G matrix (here GG) for dip-slip parts.

for i in range(n_obs_des):
    ob_des = [x_tmp_des[i], y_tmp_des[i], 0] # at surface
    for r in range(nrow): 
        for c in range(ncol): 
            al1 = c*dx  
            al2 = al1 + dx
            aw1 = -r*dy  
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob_des, depth_top, dip, [al1, al2], [aw1, aw2], [0, 1, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 

            
            u1 = u[0] * 1.e6 # convert to mm
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation
 
            los_d_sim = u3 * np.cos(inc_d) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_d) * np.sin(inc_d))
            GG_des[i][ncol*r+c] = los_d_sim

#concatenate those G into one G           
G_new_des = np.hstack((G_des,GG_des))   # its size should be (n_obs * 2 nrow ncol)

############## END of G for DESCENDING

## G for Ascending

G_asc = np.zeros((n_obs_asc, nrow*ncol)) # for strike-slip motion
GG_asc = np.zeros((n_obs_asc, nrow*ncol)) # for dip-slip motion

for i in range(n_obs_asc):
    ob_asc = [x_tmp_asc[i], y_tmp_asc[i], 0] # at surface
    for r in range(nrow): 
        for c in range(ncol): 
            al1 = c*dx  
            al2 = al1 + dx
            aw1 = -r*dy  
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob_asc, depth_top, dip, [al1, al2], [aw1, aw2], [1, 0, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 
            #print (u)
            
            u1 = u[0] * 1.e6 
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation
 
            los_a_sim = u3 * np.cos(inc_a) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_a) * np.sin(inc_a))
            G_asc[i][ncol*r+c] = los_a_sim

# G matrix (here GG) for dip-slip parts.

for i in range(n_obs_asc):
    ob_asc = [x_tmp_asc[i], y_tmp_asc[i], 0] # at surface
    for r in range(nrow): 
        for c in range(ncol): 
            al1 = c*dx  
            al2 = al1 + dx
            aw1 = -r*dy  
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob_asc, depth_top, dip, [al1, al2], [aw1, aw2], [0, 1, 0]) ## [0, 0, 1] is for tensile dislocation. For slip dislocaiont, set it to [1, 0, 0]; for dip dislocaiont set it to [0, 1, 0] 

            
            u1 = u[0] * 1.e6 # convert to mm
            u2 = u[1] * 1.e6 
            u3 = u[2] * 1.e6 
            ## convert to LOS deformation
 
            los_a_sim = u3 * np.cos(inc_a) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_a) * np.sin(inc_a))
            GG_asc[i][ncol*r+c] = los_a_sim

#concatenate those G into one G           
G_new_acs = np.hstack((G_asc,GG_asc))   # its size should be (n_obs * 2 nrow ncol)

#### Merging Greeb functions from different des and asc

G_new = np.vstack((G_new_des,G_new_acs))  # should be (n_obs_des + n_obs_asc)*(2*nrow*ncol)

                       
### Create roughness/smoothness matrix I
### our constrain: e.g., v1-2*v2+v3=0

I = np.zeros((nrow*ncol, nrow*ncol)) # for strike-slip motion
I1 = np.zeros((nrow*ncol, nrow*ncol)) # for dip-slip motion


for r in range(nrow*ncol):
    for c in range(nrow*ncol):
        if (r==c):
            I[r][c] = -2
        elif (c>r):
            I[r][r+1] = 1
        elif (c<r):
            I[r][r-1] = 1



for r in range(nrow*ncol):
    for c in range(nrow*ncol):
        if (c==r*ncol):
            I[c][c-1] = 0
            I[c-1][c] = 0
            
        
###### finished            

####the other section (vertical) I1


for r in range(nrow*ncol):
    for c in range(nrow*ncol):
        if (r==c):
            I1[r][c] = -2
        elif (c<r) and (r>=ncol):
            I1[r][r-ncol] = 1
        elif (c>r) and (c+ncol<=nrow*ncol):
            I1[r][r+ncol] = 1

####finished vertical part

I_ss = np.vstack((I,I1))
I_ds = np.vstack((I,I1))

zero1 = np.zeros((2*nrow*ncol, nrow*ncol))     

I_new_ss = np.hstack((I_ss, zero1)) #smoothness for strike-slip

I_new_ds = np.hstack((zero1, I_ds)) #smoothness for dip-slip
I_new = np.vstack((I_new_ss, I_new_ds)) #smoothness matrix 

#initializing the bounding value for LS like step 1

lb = np.zeros(2*nrow*ncol)
ub = np.zeros(2*nrow*ncol)


for p in range(nrow*ncol): # ub = [0.002,.....0.002,0,0,0...0] #lb = [0,0,0....0, -0.0005,....-0.0005]
    ub[p] = 0.002
    
for k in range(nrow*ncol,2*nrow*ncol):
    lb[k] = -0.0002

######################    

C = np.zeros((n_obs + 4*nrow*ncol, 2*nrow*ncol))
for i in range(n_obs):
    for j in range(2*nrow*ncol):
        C[i][j] = G_new[i][j]

for i in range(4*nrow*ncol):
    for j in range(2*nrow*ncol):
        C[i+n_obs][j] = e * I_new[i][j]



D =[0.0]*(n_obs + 4*nrow*ncol)
for i in range(n_obs):
    D[i] = obs_los_utm[i]



tmp = lsq_linear(C, D, bounds=(lb, ub), lsmr_tol='auto', max_iter=200, verbose=1)

print ('misfit: ', norm(tmp.fun)) #it was tmp[1] or tmp[3] look at the manual of scipy


# we have 2 types of motion here. disl1 and disl2
disl1 =np.zeros((nrow, ncol))
disl2 =np.zeros((nrow, ncol))

for i in range(nrow):
    for j in range(ncol):
        disl1[i][j] = tmp.x[ncol*i+j] / 1000    #km to km/1000 before multiplication / 1000



## save disl1 to file
np.savetxt('ss_disl1_nevada_desc+asc_e%.2e.dat'%(e), disl1, delimiter='	', fmt='%.3e')

for i in range(nrow):
    for j in range(ncol):
        disl2[i][j] = tmp.x[ncol*i+j+(nrow*ncol)] / 1000   #km to km/1000 before multiplication / 1000

## save disl2 to file        
np.savetxt('ds_disl2_nevada_desc+asc_e%.2e.dat'%(e), disl2, delimiter='	', fmt='%.3e')

#calculating the depth for each pathes

z_patch=[]
for r in range(nrow):
    for c in range(ncol):
        z_patch_value = (depth_top + (r * dy * np.sin(dip_rad)))  
        z_patch.append(z_patch_value)
        

z_patch1 = [element * -1 for element in z_patch]


You have now two files for strike-slip and dip-slip motions on the fault for each patches. you can easily plot the slip on the fault.First, you can measure the total slip.

In [10]:
### Converting the results to meter

for i in range(nrow):
    for j in range(ncol):
        disl1[i][j] = disl1[i][j] * 1.0e6


for i in range(nrow):
    for j in range(ncol):
        disl2[i][j] = disl2[i][j] * 1.0e6



In [24]:
##### plot inverted strike-slip for each patch

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
xedge = [0] * (ncol)       
yedge = [0] * (nrow)        

for i in range(len(xedge)):
    xedge[i] = dx*i     
for i in range(len(yedge)):
    yedge[i] = z_patch1[(i*ncol)]    



normal = plt.Normalize(disl1.min(), disl1.max())
disl1_scale = np.zeros((nrow,ncol))
off = disl1.min()
scale = disl1.max() - disl1.min()
for i in range(nrow):
    for j in range(ncol):
        disl1_scale[i][j] = (disl1[i][j]-off)/scale

### plot
import matplotlib.colorbar as cbar
 
fig, ax = plt.subplots(1, sharex=True, figsize=(10,8)) #it was 1
fig.subplots_adjust(hspace=0)
cmap = plt.cm.rainbow #rainbow #seismic

yedges = np.zeros((ncol,nrow))
for i in range(ncol):
    for j in range(nrow):
        yedges[i][j] = yedge[nrow-j-1]
        
        
        

for i in range(len(xedge)-1):
    for j in range(yedges.shape[1]-1):
        ax.vlines(xedge[i],yedges[i][j],yedges[i][j+1],linestyle='-',linewidth=0.5)
        ax.hlines(yedges[i][j],xedge[i],xedge[i+1],linestyle='-',linewidth=0.5)
        ax.vlines(xedge[i+1],yedges[i][j],yedges[i][j+1],linestyle='-',linewidth=0.5)
        ax.hlines(yedges[i][j+1],xedge[i],xedge[i+1],linestyle='-',linewidth=0.5)
        ax.fill_between([xedge[i],xedge[i+1]],yedges[i][j],yedges[i][j+1], facecolor=cmap(disl1_scale[nrow-1-j][i])) #disl3_scale


major_ticks = np.arange(0, 36, 5) 
ax.set_xticks(major_ticks)
ax.set_aspect('equal')
ax.set_xlabel("Along strike (km)")
ax.set_ylabel("Depth (km)")


bbox_args = dict(boxstyle="square , pad = 0.3", fc="1", ec="black", lw=2)  #boxstyle="darrow , pad = 0.3' or square, round
#arrow_args = dict(arrowstyle="->")
ax.annotate('Projected surface rupture area', xy=(2, 0),
             xytext=(8, 5), textcoords='offset points',
             ha="left", va="bottom", fontsize = 11,
             bbox=bbox_args)
            
          

cax, _ = cbar.make_axes(ax,fraction=0.023, shrink=1) 
cb = cbar.ColorbarBase(cax, cmap, norm=normal)


cb.set_label('Strike-slip motion (m)',labelpad = 6)


fig.savefig('SS_disl1_e%.2e_modeled_grid.png'%(e), dpi=300,bbox_inches='tight')




In [25]:
##### plot inverted dip-slip for each patch


for i in range(n_obs):
    obs_los_utm[i] = obs_los_utm[i] / 1000

#plot dip-slip

xedge = [0] * (ncol)       
yedge = [0] * (nrow)        

for i in range(len(xedge)):
    xedge[i] = dx*i                  
for i in range(len(yedge)):
    yedge[i] = z_patch1[(i*ncol)]    



normal = plt.Normalize(disl2.min(), disl2.max())
disl2_scale = np.zeros((nrow,ncol))
off = disl2.min()
scale = disl2.max() - disl2.min()
for i in range(nrow):
    for j in range(ncol):
        disl2_scale[i][j] = (disl2[i][j]-off)/scale

### plot
import matplotlib.colorbar as cbar

fig, ax = plt.subplots(1, sharex=True, figsize=(10,8)) #it was 1
fig.subplots_adjust(hspace=0)
cmap = plt.cm.rainbow
cmap1 = cmap.reversed()

yedges = np.zeros((ncol,nrow))
for i in range(ncol):
    for j in range(nrow):
        yedges[i][j] = yedge[nrow-j-1]
        
        
        

for i in range(len(xedge)-1):
    for j in range(yedges.shape[1]-1):
        ax.vlines(xedge[i],yedges[i][j],yedges[i][j+1],linestyle='-',linewidth=0.5)
        ax.hlines(yedges[i][j],xedge[i],xedge[i+1],linestyle='-',linewidth=0.5)
        ax.vlines(xedge[i+1],yedges[i][j],yedges[i][j+1],linestyle='-',linewidth=0.5)
        ax.hlines(yedges[i][j+1],xedge[i],xedge[i+1],linestyle='-',linewidth=0.5)
        ax.fill_between([xedge[i],xedge[i+1]],yedges[i][j],yedges[i][j+1], facecolor=cmap1(disl2_scale[nrow-1-j][i])) #disl3_scale

ax.set_aspect('equal')
ax.set_xlabel("Along strike (km)")
ax.set_ylabel("Depth (km)")
major_ticks = np.arange(0, 36, 5) 
ax.set_xticks(major_ticks)


bbox_args = dict(boxstyle="square , pad = 0.3", fc="1", ec="black", lw=2)  #boxstyle="darrow , pad = 0.3' or square, round

ax.annotate('Projected surface rupture area', xy=(2, 0),
             xytext=(8, 5), textcoords='offset points',
             ha="left", va="bottom", fontsize = 11,
             bbox=bbox_args)


## add colorbar
cax, _ = cbar.make_axes(ax,fraction=0.023, shrink=1) 
cb = cbar.ColorbarBase(cax, cmap1, norm=normal)

cb.set_label('dip-slip motion (m)',labelpad = 6)


fig.savefig('DS_disl2_e%.2e_modeled_grid.png'%(e), dpi=300, bbox_inches='tight')


You can measure total displacement (ss_ds) and moment and geodetic moment magnitude.

In [13]:

ss_ds =np.zeros((nrow, ncol))

for i in range(nrow):
    for j in range(ncol):
        ss_ds[i][j] = math.sqrt(disl1[i][j]**2 + disl2[i][j]**2)  #meter
        
sum_ss_ds = np.sum(ss_ds)   #meter
rigidity = 3*10**10                 #shear modulus = 32 GPa in crust
moment = sum_ss_ds * dx * dy * 1000 * 1000 * rigidity * 10**7 # m*m*m* N/m2 it will be m_o

momen_mag = (2/3 * math.log10(moment)) - 10.7  


In [26]:
import matplotlib.ticker as ticker

##### plot inverted total motion for each patch 
xedge = [0] * (ncol)       
yedge = [0] * (nrow)       

for i in range(len(xedge)):
    xedge[i] = dx*i                  
for i in range(len(yedge)):
    yedge[i] = z_patch1[(i*ncol)]    




normal = plt.Normalize(ss_ds.min(), ss_ds.max())
ss_ds_scale = np.zeros((nrow,ncol))
off = ss_ds.min()
scale = ss_ds.max() - ss_ds.min()
for i in range(nrow):
    for j in range(ncol):
        ss_ds_scale[i][j] = (ss_ds[i][j]-off)/scale

### plot
import matplotlib.colorbar as cbar

fig, ax = plt.subplots(1, sharex=True, figsize=(10,8)) #it was 1
fig.subplots_adjust(hspace=0)


yedges = np.zeros((ncol,nrow))
for i in range(ncol):
    for j in range(nrow):
        yedges[i][j] = yedge[nrow-j-1]
        
       


for i in range(len(xedge)-1):
    for j in range(yedges.shape[1]-1):
        ax.vlines(xedge[i],yedges[i][j],yedges[i][j+1],linestyle=(0,(0,10)))
        ax.hlines(yedges[i][j],xedge[i],xedge[i+1],linestyle=(0,(0,10)))
        ax.vlines(xedge[i+1],yedges[i][j],yedges[i][j+1],linestyle=(0,(0,10)))
        ax.hlines(yedges[i][j+1],xedge[i],xedge[i+1],linestyle=(0,(0,10)))
        ax.fill_between([xedge[i],xedge[i+1]],yedges[i][j],yedges[i][j+1], facecolor=cmap(ss_ds_scale[nrow-1-j][i])) #disl3_scale
        
        
#contours
levels = np.arange(0.2,1.5,0.3)
CS = ax.contour(xedge, yedge, ss_ds, colors = 'k', levels= levels) # OR cmap = cmap 
ax.clabel(CS, inline=True, fontsize=10, fmt="%2.1f", use_clabeltext=True)


arrays_arrow_x = range(7,30,1)     #[7-29]col
arrays_arrow_y = range(5,16,1)     #[5-15]row


nrow2 = range(1,18) 
ncol2 = range(5,35) 

for i in (nrow2):
    for j in (ncol2):
        ax.arrow(xedge[j],yedge[i]-0.4, disl1[i][j]/np.max(disl1), disl2[i][j], length_includes_head =True, head_width = 0.07, head_length = 0.07, color = 'k', head_starts_at_zero = True)
        


ax.set_aspect('equal')
ax.set_xlabel("Along strike (km)")
ax.set_ylabel("Depth (km)")


bbox_args = dict(boxstyle="square , pad = 0.3", fc="1", ec="black", lw=2)  #boxstyle="darrow , pad = 0.3' or square, round
#arrow_args = dict(arrowstyle="->")
ax.annotate('Projected surface rupture area', xy=(2, 0),
             xytext=(8, 5), textcoords='offset points',
             ha="left", va="bottom", fontsize = 11,
             bbox=bbox_args)

# drwaing arrow legend
ax.arrow(41,1,1/np.max(disl1),0, length_includes_head =True, head_width = 0.08, head_length = 0.08, color = 'k', head_starts_at_zero = True, clip_on = False)
ax.text(41.5,1.6,'1(m)', horizontalalignment='center', verticalalignment='center',fontsize=11)


## add colorbar
cax, _ = cbar.make_axes(ax,fraction=0.023, shrink=1) 
cb = cbar.ColorbarBase(cax, cmap, norm=normal)


cb.set_label('Total motion (m)',labelpad = 6)


fig.savefig('SS_DS_total_e%.2e_modeled_grid.png'%(e), dpi=300, bbox_inches='tight')




In [17]:
import numpy as np
from okada_wrapper import dc3dwrapper

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


### explanation for units
## Distance: km

## Geometry of patches
## x0, y0 is the origin (left, bottom)
x0 = 404
y0 = 4222
dx = 1.0 # spacing along X/row
dy = 1.0 # spacing along Y/column
nrow = 20 # number of row dip 
ncol = 40 # number of column strike

## patch parameters
alpha = 0.6666666
depth_top = 0 # km  #it will be 1.2 to 20 km for instance
dip = 80
strike = 76 # degree
rake = 0 # degree


e = 6.16e+05

a = -1.0*(strike-90.)*np.pi/180. # rotation angle



## read the inverted tensile disl3

disl1 = np.loadtxt("ss_disl1_nevada_desc+asc_e%.2e.dat"%(e)) #default data type is float
disl2 = np.loadtxt("ds_disl2_nevada_desc+asc_e%.2e.dat"%(e)) #default data type is float

## read InSAR DESCENDING LOS observation
f_des = open('descending_example_nevada.dat','r')   
line = f_des.readline().strip() # skip the first line
obs_x_utm_des = []
obs_y_utm_des = []
obs_los_utm_des = []

while True:
    line = f_des.readline().strip()
    if line=='':
        #print ('break')
        break
    raw = line.split('	')
    obs_x_utm_des.append(float(raw[0]))	
    obs_y_utm_des.append(float(raw[1]))
    obs_los_utm_des.append(float(raw[2]))
f_des.close()

## read InSAR ASCENDING LOS observation
f_asc = open('ascending_example_nevada.dat','r')   
line = f_asc.readline().strip() # skip the first line
obs_x_utm_asc = []
obs_y_utm_asc = []
obs_los_utm_asc = []

while True:
    line = f_asc.readline().strip()
    if line=='':
        #print ('break')
        break
    raw = line.split('	')
    obs_x_utm_asc.append(float(raw[0]))	
    obs_y_utm_asc.append(float(raw[1]))
    obs_los_utm_asc.append(float(raw[2]))
f_asc.close()

######################## Concatenating vectors
obs_x_utm = np.concatenate((obs_x_utm_des, obs_x_utm_asc))
obs_y_utm = np.concatenate((obs_y_utm_des, obs_y_utm_asc))
obs_los_utm = np.concatenate((obs_los_utm_des, obs_los_utm_asc))

n_obs_des = len(obs_los_utm_des)
n_obs_asc = len(obs_los_utm_asc)
n_obs = len(obs_los_utm)


#should convert mm to m of observation data

for i in range(n_obs):
    obs_los_utm[i] = obs_los_utm[i] / 1000

x_tmp=[] #converted coordinates to okada coordinate
y_tmp=[]

for i in range(n_obs):
    xxmp = (obs_x_utm[i] - x0)*np.cos(-a) - (obs_y_utm[i] - y0)*np.sin(-a)
    yymp = (obs_x_utm[i] - x0)*np.sin(-a) + (obs_y_utm[i] - y0)*np.cos(-a)
    x_tmp.append(xxmp)
    y_tmp.append(yymp)
    

## convert disp to LOS. use the equation in Hu, J., Li, Z., et al., 2012
#it is descending
inc_d = 34.0 * np.pi/180. # rad
heading_d = -166.71 * np.pi/180.0

#it is ascending
inc_a = 3.397904225855917e+01 * np.pi/180. # rad
heading_a = -1.324459955550515e+01 * np.pi/180.0

######

los_sim = [0.] * n_obs
los_sim_d = [0.] * n_obs_des
los_sim_a= [0.] * n_obs_asc

### save result to file
f_out = open('simulated_UTM_Ux_Uy_Uz_los_desc+asc.dat', 'w')
f_out.write("Easting(km)\tNorthing(km)\tUx(mm)\tUy(mm)\tUz(mm)\tLOS_simulated(mm)\tLOS_observed(mm)\n")

for i in range(n_obs):
    ob = [x_tmp[i], y_tmp[i], 0] 
    u1 = 0.
    u2 = 0.
    u3 = 0.
    for r in range(nrow):
        for c in range(ncol):
            al1 = c*dx
            al2 = al1 + dx
            aw1 = -r*dy
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob, depth_top, dip, [al1, al2], [aw1, aw2], [disl1[r][c], disl2[r][c], 0])  #disl are in km/1000
            u1 = u[0] * 1.e6 + u1
            u2 = u[1] * 1.e6 + u2
            u3 = u[2] * 1.e6 + u3
            
    
    if (i<(n_obs_des)):
        los_sim[i] = u3 * np.cos(inc_d) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_d) * np.sin(inc_d))

        
    elif (i>(n_obs_des)):
        los_sim[i] = u3 * np.cos(inc_a) \
                        - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_a) * np.sin(inc_a))

        
        
 

    f_out.write("%f\t%f\t%f\t%f\t%f\t%f\t%f\n" % (obs_x_utm[i], obs_y_utm[i], u1, u2, u3, los_sim[i], obs_los_utm[i]))   

f_out.close()


### Only Descending

for i in range(n_obs_des):
    ob = [x_tmp_des[i], y_tmp_des[i], 0] 
    u1 = 0.
    u2 = 0.
    u3 = 0.
    for r in range(nrow):
        for c in range(ncol):
            al1 = c*dx
            al2 = al1 + dx
            aw1 = -r*dy
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob, depth_top, dip, [al1, al2], [aw1, aw2], [disl1[r][c], disl2[r][c], 0])  #disl are in km/1000
            u1 = u[0] * 1.e6 + u1
            u2 = u[1] * 1.e6 + u2
            u3 = u[2] * 1.e6 + u3
            
    los_sim_d[i] = u3 * np.cos(inc_d) - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_d) * np.sin(inc_d))

### only Ascending

for i in range(n_obs_asc):
    ob = [x_tmp_asc[i], y_tmp_asc[i], 0] 
    u1 = 0.
    u2 = 0.
    u3 = 0.
    for r in range(nrow):
        for c in range(ncol):
            al1 = c*dx
            al2 = al1 + dx
            aw1 = -r*dy
            aw2 = aw1 - dy

            success,u,grad_u = dc3dwrapper(alpha, ob, depth_top, dip, [al1, al2], [aw1, aw2], [disl1[r][c], disl2[r][c], 0])  #disl are in km/1000
            u1 = u[0] * 1.e6 + u1
            u2 = u[1] * 1.e6 + u2
            u3 = u[2] * 1.e6 + u3
            
    los_sim_a[i] = u3 * np.cos(inc_a) - (u1*(np.cos(a)) - u2*(np.sin(a))) * (np.cos(heading_a) * np.sin(inc_a))




In [27]:
#clipping descending data to zoom in the fault area

obs_x_des_clip =[]
obs_y_des_clip =[]
obs_los_des_clip =[]
obs_simulated_des_clip =[]
for i in range(n_obs_des):
    if obs_x_utm_des[i] >= 400 and obs_x_utm_des[i] <= 450 and obs_y_utm_des[i] >= 4210 and obs_y_utm_des[i] <= 4240:
        obs_x_des_clip.append(obs_x_utm_des[i])
        obs_y_des_clip.append(obs_y_utm_des[i])
        obs_los_des_clip.append(obs_los_utm_des[i])
        obs_simulated_des_clip.append(los_sim_d[i])
        
for i in range(len(obs_los_des_clip)):
    obs_los_des_clip[i] = obs_los_des_clip[i] / 1000

    

los_res_dd = []    ### residuals
for i in range(len(obs_los_des_clip)):
    resid_value = (obs_los_des_clip[i] - obs_simulated_des_clip[i]) 
    los_res_dd.append(resid_value)



### plot observation, simulation, residuals / scatter style
fig = plt.figure(figsize=(14,10))
gs = gridspec.GridSpec(1,3)
gs.update(wspace=0.3, hspace=0.0)

cm = plt.cm.get_cmap('rainbow') #cmap=plt.cm.coolwarm,
# vmin = np.min(obs_los_utm)
# vmax = np.max(obs_los_utm)

for i in range(3):
    ax = fig.add_subplot(gs[i])
    if i == 0:
        sc1 = ax.scatter(obs_x_des_clip, obs_y_des_clip, marker='o', c=obs_los_des_clip, s=6, edgecolors='none', cmap=cm) #vmin=vmin, vmax=vmax
        ax.set_title("Observation")
        ax.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 
        
    elif i ==1:
        sc2 = ax.scatter(obs_x_des_clip, obs_y_des_clip, marker='o', c=obs_simulated_des_clip, s=6, edgecolors='none', cmap=cm)#vmin=vmin, vmax=vmax
        ax.set_title("Simulation")
        ax.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 

    elif i ==2:
        sc3 = ax.scatter(obs_x_des_clip, obs_y_des_clip, marker='o', c=los_res_dd, s=6, edgecolors='none', cmap=cm)
        ax.set_title("Residual")
        ax.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 

    ax.set_aspect('equal')
    ax.set_xlabel("Easting (km)")
    ax.set_ylabel("Northing (km)")




    
    
## add colorbar
cbaxes=fig.add_axes([0.91,0.41,0.02,0.18])
cbar = plt.colorbar(sc1, cax=cbaxes, orientation='vertical') #,ticks=np.arange(vmin, vmax+0.001, 10)
cbar.ax.locator_params(nbins=5) #,ticks=np.arange(vmin, vmax+0.001, 10)
cbar.ax.tick_params(labelsize=12)  # colorbar tick
#cbar.set_label('LOS deformation (m)', fontsize=12, labelpad = 6) # colorbar label
cbar.ax.set_title('(m)')

fig.savefig('Descending_residual_e%.2e.png'%(e), dpi=300, format='png',bbox_inches='tight')

In [28]:
#clipping ascending data

obs_x_asc_clip =[]
obs_y_asc_clip =[]
obs_los_asc_clip =[]
obs_simulated_asc_clip =[]
for i in range(n_obs_asc):
    if obs_x_utm_asc[i] >= 400 and obs_x_utm_asc[i] <= 450 and obs_y_utm_asc[i] >= 4210 and obs_y_utm_asc[i] <= 4240:
        obs_x_asc_clip.append(obs_x_utm_asc[i])
        obs_y_asc_clip.append(obs_y_utm_asc[i])
        obs_los_asc_clip.append(obs_los_utm_asc[i])
        obs_simulated_asc_clip.append(los_sim_a[i])
        
for i in range(len(obs_los_asc_clip)):
    obs_los_asc_clip[i] = obs_los_asc_clip[i] / 1000
    
#calculation of ascending residuals for clipped ascending
los_res_aa = []
for i in range(len(obs_los_asc_clip)):
    resid_value = (obs_los_asc_clip[i] - obs_simulated_asc_clip[i]) 
    los_res_aa.append(resid_value)
    

from matplotlib import ticker

### plot observation, simulation, residuals / scatter style
fig = plt.figure(figsize=(14,10))
gs = gridspec.GridSpec(1,3)
gs.update(wspace=0.3, hspace=0.0)

cm = plt.cm.get_cmap('rainbow') 

for i in range(3):
    ax = fig.add_subplot(gs[i])
    if i == 0:
        sc1 = ax.scatter(obs_x_asc_clip, obs_y_asc_clip, marker='o', c=obs_los_asc_clip, s=6, edgecolors='none', cmap=cm) #vmin=vmin, vmax=vmax
        ax.set_title("Observation")

        ax.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 

    elif i ==1:
        sc2 = ax.scatter(obs_x_asc_clip, obs_y_asc_clip, marker='o', c=obs_simulated_asc_clip, s=6, edgecolors='none', cmap=cm)#vmin=vmin, vmax=vmax
        ax.set_title("Simulation")
        ax.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 
    else:
        sc3 = ax.scatter(obs_x_asc_clip, obs_y_asc_clip, marker='o', c=los_res_aa, s=6, edgecolors='none', cmap=cm)
        ax.set_title("Residual")
        ax.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4)) 
    ax.set_aspect('equal')
    ax.set_xlabel("Easting (km)")
    ax.set_ylabel("Northing (km)")
   
    
## add colorbar

cbaxes=fig.add_axes([0.91,0.41,0.02,0.18])
cbar = plt.colorbar(sc1, cax=cbaxes, orientation='vertical') 
cbar.ax.locator_params(nbins=4)
cbar.ax.tick_params(labelsize=12)  
cbar.ax.set_title('(m)')

fig.savefig('Ascending_residual_e%.2e.png'%(e), dpi=300, format='png',bbox_inches='tight')